In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
import psycopg2
import time

In [21]:
link = 'http://www.whosampled.com/Usher/OMG/'
html = requests.get(link).content
soup = BeautifulSoup(html, 'html.parser')
print soup


<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge; chrome=1" http-equiv="X-UA-Compatible"/><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o?o:n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({QJf3ax:[function(e,n){function t(e){function n(n,t,a){e&&e(n,t,a),a||(a={});for(var u=c(n),f=u.length,s=i(a,o,r),p=0;f>p;p++)u[p].apply(s,t);return s}function a(e,n){f[e]=c(e).concat(n)}function c(e){return f[e]||[]}function u(){return t(n)}var f={};return{on:a,emit:n,create:u,listeners:c,_events:f}}function r(){return{}}var o="nr@context",i=e("gos");n.exports=t()},{gos:"7eSDFh"}],ee:[function(e,n){n.exports=e("QJ

In [11]:
def song_data(soup):
    url_base = 'http://www.whosampled.com'
    song = {}
    
    song['name'] = soup.select('h1[itemprop="name"]')[0].text
    print song['name']
    song['artists'] = [x.text for x in soup.select('h2 a')]
    song['artist_links'] = [url_base + x['href'] for x in soup.select('h2 a')]
    song['album'] = soup.select('.trackReleaseDetails h3')[0].text
    song['record_label'] = re.split('; |, |\*|\n', soup.select('.trackReleaseDetails h4')[0].text)
    song['year'] = soup.select('.trackReleaseDetails')[0].contents[4].strip()
    if len(soup.select('.trackReleaseDetails a')) > 0:
        song['producers'] = soup.select('.trackReleaseDetails a')[0].text
    else:
        song['producers'] = ''
    song['relationships'] = sample_data(soup)
    
    return song

In [12]:
def sample_data(soup):
    song_sample_data = {}
    for header in soup.select('.sectionHeader'):
        if header.text.find('samples') != -1:
            ul = header.find_next_sibling('ul')
            song_sample_data['sampled_from_song'] = [x.text for x in ul.select('.trackName')]
            song_sample_data['sampled_from_song_url'] = [x['href'] for x in ul.select('.trackName')]
            song_sample_data['sampled_from_artist'] = [x.text for x in ul.select('.trackArtist a')]
            song_sample_data['sampled_from_artist_url'] = [x['href'] for x in ul.select('.trackArtist a')]
        elif header.text.find('sampled') != -1:
            ul = header.find_next_sibling('ul')
            song_sample_data['sampled_by_song'] = [x.text for x in ul.select('.trackName')]
            song_sample_data['sampled_by_song_url'] = [x['href'] for x in ul.select('.trackName')]
            song_sample_data['sampled_by_artist'] = [x.text for x in ul.select('.trackArtist a')]
            song_sample_data['sampled_by_artist_url'] = [x['href'] for x in ul.select('.trackArtist a')]
        elif header.text.find('covered') != -1:
            ul = header.find_next_sibling('ul')
            song_sample_data['covered_by_song'] = [x.text for x in ul.select('.trackName')]
            song_sample_data['covered_by_song_url'] = [x['href'] for x in ul.select('.trackName')]
            song_sample_data['covered_by_artist'] = [x.text for x in ul.select('.trackArtist a')]
            song_sample_data['convered_by_artist_url'] = [x['href'] for x in ul.select('.trackArtist a')]
        elif header.text.find('remixed') != -1:
            ul = header.find_next_sibling('ul')
            song_sample_data['remixed_by_song'] = [x.text for x in ul.select('.trackName')]
            song_sample_data['remixed_by_song_url'] = [x['href'] for x in ul.select('.trackName')]
            song_sample_data['remixed_by_artist'] = [x.text for x in ul.select('.trackArtist a')]
            song_sample_data['remixed_by_artist_url'] = [x['href'] for x in ul.select('.trackArtist a')]
    return song_sample_data

In [22]:
song = song_data(soup)

print song['album']
print song['record_label']
print song['year']
print song['producers']
print song

OMG
[u'Usher', u'will.i.am'] Raymond v. Raymond [u'LaFace'] 2010 will.i.am
Raymond v. Raymond
[u'LaFace']
2010
will.i.am
{'album': u'Raymond v. Raymond', 'record_label': [u'LaFace'], 'name': u'OMG', 'relationships': {'remixed_by_song_url': [u'/remix/72189/Usher-will.i.am-OMG-(Riva-Starr-Remix)-Usher-will.i.am-OMG/', u'/remix/72190/Usher-will.i.am-OMG-(Almighty-Mix)-Usher-will.i.am-OMG/'], 'sampled_by_artist_url': [u'/Girl-Talk/', u'/DJ-Earworm/', u'/DJ-Earworm/'], 'covered_by_artist': [u'Audiogroove'], 'remixed_by_artist': [u'Riva Starr', u'Usher', u'will.i.am'], 'sampled_from_artist_url': [u'/Deep-Purple/', u'/Black-Eyed-Peas/'], 'sampled_from_song': [u'Speed King (Live)', u'Boom Boom Pow'], 'sampled_by_artist': [u'Girl Talk', u'DJ Earworm', u'DJ Earworm'], 'remixed_by_song': [u'OMG (Riva Starr Remix)', u'OMG (Almighty Mix)'], 'covered_by_song': [u'OMG (Oh My Gosh)'], 'sampled_from_artist': [u'Deep Purple', u'Black Eyed Peas'], 'sampled_by_song_url': [u'/sample/68913/Girl-Talk-Down-fo

In [14]:
def get_soup(url):
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [15]:
def get_song_urls_from_artist(artist):
    url_base = 'http://www.whosampled.com'
    song_urls = []
    url = url_base + '/' + artist
    soup = get_soup(url)
    pages = int(soup.select('.page')[-1].text)
    for pg_num in xrange(pages):
        url = url_base + '/' + artist + '/?sp=' + str(pg_num+1)
        soup = get_soup(url)
        current_pg_urls = [url_base + song['href'] for song in soup.select('.trackName a[itemprop="url"]')]
        song_urls += current_pg_urls
    return song_urls

In [16]:
get_song_urls_from_artist('Drake')

[u'http://www.whosampled.com/Drake/0-to-100-the-Catch-Up/',
 u'http://www.whosampled.com/Drake/Pound-CakeParis-Morton-Music-2/',
 u'http://www.whosampled.com/Nicki-Minaj/Truffle-Butter/',
 u'http://www.whosampled.com/Kendrick-Lamar/Poetic-Justice/',
 u'http://www.whosampled.com/Drake/Started-From-the-Bottom/',
 u'http://www.whosampled.com/Drake/Know-Yourself/',
 u'http://www.whosampled.com/Drake/Take-Care/',
 u'http://www.whosampled.com/Drake/Legend/',
 u'http://www.whosampled.com/Drake/6-God/',
 u'http://www.whosampled.com/Drake/Fancy/',
 u'http://www.whosampled.com/The-Game/100/',
 u'http://www.whosampled.com/Drake/Jungle/',
 u'http://www.whosampled.com/A$AP-Rocky/Fuckin%27-Problems/',
 u'http://www.whosampled.com/Drake/Miss-Me/',
 u'http://www.whosampled.com/Drake/Cameras-Good-Ones-Go-Interlude/',
 u'http://www.whosampled.com/DJ-Khaled/I%27m-on-One/',
 u'http://www.whosampled.com/PARTYNEXTDOOR/Over-Here/',
 u'http://www.whosampled.com/Drake/How-Bout-Now/',
 u'http://www.whosampled.c

In [ ]:
for header in soup.select('.sectionHeader'):
    print header.text
    ul = header.find_next_sibling('ul')
    print ul.select('.trackName')[0].text

In [17]:
def scrape_Drake(artist):
    all_data = []
    artist = '-'.join(artist.split())
    urls = get_song_urls_from_artist(artist)
    for url in urls:
        soup = get_soup(url)
        all_data.append(song_data(soup))
    return all_data

In [18]:
data = scrape_Drake('Chance the Rapper')

Favorite Song
[u'Chance the Rapper', u'Childish Gambino'] Acid Rap [u''] 2013 Nate Fox
Lost
[u'Chance the Rapper', u'Noname Gypsy'] Acid Rap [u''] 2013 Nate Fox
NaNa
[u'Chance the Rapper', u'Action Bronson'] Acid Rap [u''] 2013 brandUn DeShay
Juice
[u'Chance the Rapper'] Acid Rap [u''] 2013 Nate Fox
Pusha Man
[u'Chance the Rapper', u'Nate Fox', u'Lili K'] Acid Rap [u'-'] 2013 CeeJ
Everybody's Something
[u'Chance the Rapper', u'Saba', u'BJ the Chicago Kid'] Acid Rap [u''] 2013 DJ Ozone
Good Ass Intro
[u'Chance the Rapper'] Acid Rap [u''] 2013 
Wendy N Becky
[u'Joey Bada$$', u'Chance the Rapper']  [u'Cinematic Music Group'] 2013 Thelonious Martin
Chain Smoker
[u'Chance the Rapper'] Acid Rap [u'Mixtape'] 2013 Nate Fox
Long Time
[u'Chance the Rapper'] 10 Day [u'SaveMoney'] 2012 
22 Offs
[u'Chance the Rapper'] 10 Day [u'Not on Label'] 2012 Flying Lotus
14,400 Minutes
[u'Chance the Rapper'] 10 Day [u'Not on Label'] 2012 DJ SuchNSuch
Everything's Good (Good Ass Outro)
[u'Chance the Rapper'] A

In [20]:
len(data)

48

# Data architecture

###Table Artist:
artist_url - string
artist_name - string

###Table Producer:
producer_url - string
producer_name - string

###Table Song:
song_url - string
song_name - string
album_name - string
label - string
year - int
main_genre - string

###Table ArtistToSong:
artist_url - string
song_url - string

###Table ProducerToSong:
producer_url - string
song_url - string

###Table SongRelationship:
song1 - string
song2 - string
relationship - int (1 for sampled from, 2 for sampled by, 3 for cover of, 4 for covered by, 5 for remix of, and 6 for remixed by)

In [14]:
class GetData(object):
    def __init__(self):
        self.url_base = 'http://www.whosampled.com'
        self.used_genre = set([])
        self.unused_genre = set(['/genre/Hip-Hop/', '/genre/Electronic-Dance/', '/genre/Rock-Pop/'])
        self.used_artist = set([])
        self.unused_artist = set([])

    # Create soup object
    def _get_soup(self, url):
        '''
        INPUT: url string
        OUTPUT: soup object for processing
        '''
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'html.parser')
        return soup

    # Collect all artist names based on genre
    def _get_artists_by_genre(self, genre):
        '''
        INPUT: genre string
        OUTPUT: saves artists names into SQL database
        '''
        artists = []
        url = self.url_base + genre
        genre_soup = self._get_soup(url)
        alpha_urls = [x['href'] for x in genre_soup.select('.browse-letters a')]
        for alphabet_url in alpha_urls:
            pages = int(self._get_num_pages(alphabet_url))
            decomp_url = alphabet_url.split('/')
            for page in xrange(pages):
                decomp_url[-2] = str(page + 1)
                page_url = '/'.join(decomp_url)
                artists.append(self._get_link_and_text_data(page_url, '.artistName a'))
        
        for artist in artists:
            if artist[0] in self.used_artist:
                continue
            else:
                con = None
                con = psycopg2.connect(database = 'beatlineage', user = 'gSchool', host = '/tmp')
                cur = con.cursor()
                con.autocommit = True
                cur.execute('INSERT INTO artists VALUES(%s, %s)', artist)
    
    # Collect all song names based on artist
    def _get_songs_by_artist(self, artist):
        '''
        INPUT: artist string
        OUTPUT: saves producer and song data into SQL database
        '''
        pass
    
    # Get the number of pages on a page
    def _get_num_pages(self, end_url):
        '''
        INPUT: ending url string
        OUTPUT: number of pages of request
        '''
        url = self.url_base + end_url
        page_soup = self._get_soup(url)
        num_of_pg = int(page_soup.select('.page')[-1].text)
        return num_of_pg
    
    # Extract text and link data for a given CSS tag
    def _get_link_and_text_data(self, end_url, tag):
        '''
        INPUT: ending url and CSS tag
        OUTPUT: tuple of (text, link)
        '''
        print end_url
        url = self.url_base + end_url
        page_soup = self._get_soup(url)
        text = [x.text for x in page_soup.select(tag)]
        link = [x['href'] for x in page_soup.select(tag)]
        return zip(link, text)

In [15]:
data = GetData()

In [17]:
data._get_artists_by_genre('/genre/Hip-Hop/')

/browse/artists/Hip-Hop/A/1/
/browse/artists/Hip-Hop/A/2/
/browse/artists/Hip-Hop/A/3/
/browse/artists/Hip-Hop/A/4/
/browse/artists/Hip-Hop/A/5/
/browse/artists/Hip-Hop/A/6/
/browse/artists/Hip-Hop/A/7/
/browse/artists/Hip-Hop/A/8/
/browse/artists/Hip-Hop/A/9/
/browse/artists/Hip-Hop/A/10/
/browse/artists/Hip-Hop/A/11/
/browse/artists/Hip-Hop/A/12/
/browse/artists/Hip-Hop/A/13/
/browse/artists/Hip-Hop/A/14/
/browse/artists/Hip-Hop/A/15/
/browse/artists/Hip-Hop/A/16/
/browse/artists/Hip-Hop/A/17/
/browse/artists/Hip-Hop/A/18/
/browse/artists/Hip-Hop/A/19/
/browse/artists/Hip-Hop/A/20/
/browse/artists/Hip-Hop/A/21/
/browse/artists/Hip-Hop/A/22/
/browse/artists/Hip-Hop/A/23/
/browse/artists/Hip-Hop/A/24/
/browse/artists/Hip-Hop/A/25/
/browse/artists/Hip-Hop/A/26/
/browse/artists/Hip-Hop/A/27/
/browse/artists/Hip-Hop/A/28/
/browse/artists/Hip-Hop/A/29/
/browse/artists/Hip-Hop/A/30/
/browse/artists/Hip-Hop/A/31/
/browse/artists/Hip-Hop/A/32/
/browse/artists/Hip-Hop/A/33/
/browse/artists/Hip

TypeError: not all arguments converted during string formatting

In [8]:
con = None
con = psycopg2.connect(database = 'beatlineage', user = 'gSchool', host = '/tmp')
cur = con.cursor()
con.autocommit = True

In [10]:
def create_artist_table(cur, con):
    try:
        cur.execute('CREATE TABLE artists (artist_url varchar, artist_name varchar)')
    except:
        pass
    
    artist = ('/Kanye-West/', 'Kanye West')
    
    cur.execute('INSERT INTO artists VALUES(%s, %s)', artist)

create_artist_table(cur, con)